<a href="https://colab.research.google.com/github/Dhanya-Zac/Multilingual-LLM-hallucination-test/blob/main/knowledge_dataset_creation_malayalam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

"""
Simple Malayalam CSV to JSON Converter
"""

import pandas as pd
import json

def simple_csv_to_json(csv_file='mal_2000.csv', json_file='mal_dataset.json', max_rows=15000):
    """
    Simplest possible implementation for Malayalam dataset
    """
    # Read CSV
    df = pd.read_csv(csv_file, encoding='utf-8')

    # Keep first 15000 rows (or whatever is in the file)
    df = df.iloc[:max_rows]

    # Convert to JSON (array of objects format)
    df.to_json(json_file, orient='records', indent=2, force_ascii=False)

    print(f"✓ Converted {len(df)} rows from {csv_file} to {json_file}")

def oneliner_csv_to_json(csv_file='mal_2000.csv', json_file='mal_dataset.json'):
    """Ultra-compact version"""
    pd.read_csv(csv_file, encoding='utf-8').iloc[:15000].to_json(
        json_file, orient='records', indent=2, force_ascii=False
    )

if __name__ == "__main__":
    # Convert Malayalam dataset
    simple_csv_to_json('mal_2000.csv', 'mal_dataset.json')

✓ Converted 15000 rows from mal_2000.csv to mal_dataset.json


In [3]:
import json
import random
from openai import OpenAI

In [4]:
import getpass
import os

In [5]:
!pip install rapidfuzz


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 47.1 MB/s eta 0:00:00


In [6]:
import json
from rapidfuzz import fuzz
import random
import os
import unicodedata
import re
from difflib import SequenceMatcher
from openai import OpenAI
import tiktoken
from google.colab import userdata
from tqdm import tqdm

class KnowledgeDataset:
    def __init__(self, model_name="gpt-4o-mini", dataset_path="mal_500.json", save_dir="./", dataset_name="mal_dataset"):
        # Get API key from Colab secrets
        api_key = userdata.get('gptapi')
        self.client = OpenAI(api_key=api_key)

        self.model_name = model_name
        self.dataset_path = dataset_path
        self.save_dir = save_dir
        self.dataset_name = dataset_name

        # Initialize tokenizer for the model
        # For gpt-4o-mini, we use cl100k_base encoding
        self.tokenizer = tiktoken.encoding_for_model(model_name)

        # Load dataset
        with open(dataset_path, "r", encoding="utf-8") as f:
            raw_data = json.load(f)

        self.initial_dataset = []
        for item in raw_data:
            if isinstance(item, str):
                try:
                    q, a = item.split("\nഉത്തരം:")
                    q = q.replace("ചോദ്യം:", "").strip()
                    a = a.strip()
                    if q and a and q.lower() != "nan" and a.lower() != "nan":
                        self.initial_dataset.append({"ചോദ്യം": q, "ഉത്തരം": a})
                except ValueError:
                    # Silent skip for malformed items
                    pass
            elif isinstance(item, dict):
                q = item.get("ചോദ്യം", "").strip()
                a = item.get("ഉത്തരം", "").strip()
                if q and a and q.lower() != "nan" and a.lower() != "nan":
                    self.initial_dataset.append({"ചോദ്യം": q, "ഉത്തരം": a})

        # Pre-built Malayalam few-shot examples
        self.list_good_shot = [
            "ചോദ്യം: ഫ്രാൻസിന്റെ തലസ്ഥാനം ഏതാണ്?\nഉത്തരം: പാരീസ്\n",
            "ചോദ്യം: എത്ര ഭൂഖണ്ഡങ്ങളുണ്ട്?\nഉത്തരം: 7\n",
            "ചോദ്യം: പ്രകൃതിവാതകത്തിന്റെ പ്രധാന ഘടകം ഏതാണ്?\nഉത്തരം:മീഥേൻ\n",
            "ചോദ്യം: 1999 ൽ ഓസ്ട്രേലിയയിലെ മെൽബണിൽ ആരംഭിച്ച 'മോവെംബർ' എന്ന വാർഷിക ചാരിറ്റബിൾ സംരംഭം പുരുഷന്മാരുടെ ആരോഗ്യ പ്രശ്നങ്ങൾക്ക് (പ്രത്യേകിച്ച് പ്രോസ്റ്റേറ്റ്, ടെസ്റ്റിക്കിൾ കാൻസർ) വേണ്ടിയുള്ള ബോധവൽക്കരണവും ധാരണയും ഉയർത്തുന്നു. ഇതിന്റെ ഭാഗമായി പുരുഷന്മാർ എന്താണ് ധരിക്കുന്നത്?\nഉത്തരം:മീശകൾ\n",
            "ചോദ്യം: 'ദി എറിക് ബർത്തലോമെവ്' എന്ന വെതർസ്പൂൺസ് പബ് ഏത് പട്ടണത്തിലാണ്?\nഉത്തരം:മോറെകാംബെ\n",
            "ചോദ്യം: 'റോമിയോ ആൻഡ് ജൂലിയറ്റ്' എഴുതിയത് ആരാണ്?\nഉത്തരം: വില്യം ഷേക്സ്പിയർ\n",
            "ചോദ്യം: 64 ന്റെ വർഗ്ഗമൂല്യം എന്താണ്?\nഉത്തരം: 8\n",
            "ചോദ്യം: ഏത് മൂലകത്തിനാണ് 'H' എന്ന രാസചിഹ്നം ഉള്ളത്?\nഉത്തരം: ഹൈഡ്രജൻ\n",
            "ചോദ്യം: അമേരിക്കൻ ഐക്യനാടുകളുടെ ആദ്യ പ്രസിഡന്റ് ആരായിരുന്നു?\nഉത്തരം: ജോർജ്ജ് വാഷിംഗ്ടൺ\n",
            "ചോദ്യം: കോശത്തിന്റെ പവർഹൗസ് എന്താണ്?\nഉത്തരം: മൈറ്റോകോൺഡ്രിയ\n",
            "ചോദ്യം: ഗ്രനാഡ ടെലിവിഷനുവേണ്ടി ജെറി ആൻഡേഴ്‌സൺ നിർമ്മിച്ച മൂന്നാമത്തെ പപ്പറ്റ് ടെലിവിഷൻ ഷോ എന്തായിരുന്നു 'ഫോർ ___ ഫാൾസ്'?\nഉത്തരം:ഫെതർ\n",
            "ചോദ്യം: മാർഗരിറ്റ് ഹെൻറി എഴുതിയ 'കിംഗ് ഓഫ് ദി വിൻഡ്' എന്ന കുട്ടികളുടെ പുസ്തകത്തിൽ, സ്വർണ്ണ നിറത്തിലുള്ള അറേബ്യൻ കുതിരയുടെ പേരെന്താണ്?\nഉത്തരം:ഷാം\n",
            "ചോദ്യം: പോളും ലിൻഡ മക്കാർട്ട്‌നിയും ചേർന്ന് രചിച്ച ബോണ്ട് സിനിമയിലെ തീം സോങ്ങ് ഏതാണ്?\nഉത്തരം:ലൈവ് ആൻഡ് ലെറ്റ് ഡൈ\n",
            "ചോദ്യം: മോണാലിസ ആരാണ് വരച്ചത്?\nഉത്തരം: ലിയോനാർഡോ ഡാവിഞ്ചി\n"
        ]

    # ------------------------------
    # Get token IDs for a text
    # ------------------------------
    def get_token_ids(self, text):
        """Convert text to token IDs using the model's tokenizer"""
        return self.tokenizer.encode(text)

    # ------------------------------
    # Batch generation (temperature sampling)
    # ------------------------------
    def batch_generation_with_temperature(self, prompt, n=5, max_tokens=50, temperature=0.5):
        response = self.client.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature,
            n=n,
            max_completion_tokens=max_tokens
        )
        return [choice.message.content.strip() for choice in response.choices]

    # ------------------------------
    # Greedy generation (temperature=0)
    # ------------------------------
    def greedy_generation(self, prompt, max_tokens=50):
        response = self.client.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            max_completion_tokens=max_tokens
        )
        return response.choices[0].message.content.strip()

    # ------------------------------
    # Malayalam-aware fuzzy matching
    # ------------------------------
    @staticmethod
    def partial_match(a: str, gen: str, threshold: float = 65) -> bool:
        a_norm  = unicodedata.normalize("NFC", a).strip()
        gen_norm = unicodedata.normalize("NFC", gen).strip()
        if a_norm and a_norm in gen_norm:
            return True
        a_tokens  = set(re.findall(r"\w+", a_norm))
        gen_tokens = set(re.findall(r"\w+", gen_norm))
        if a_tokens and a_tokens.issubset(gen_tokens):
            return True
        return fuzz.partial_ratio(a_norm, gen_norm) >= threshold

    # ------------------------------
    # Core dataset creation
    # ------------------------------
    def create_knowledge_dataset(self, save_every=100):
        knowledge_dataset = []
        non_knowledge_dataset = []
        else_dataset = []

        os.makedirs(self.save_dir, exist_ok=True)

        # Create progress bar with custom formatting
        pbar = tqdm(
            self.initial_dataset,
            desc="Processing dataset",
            total=len(self.initial_dataset),
            bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]'
        )

        for index, point in enumerate(pbar, 1):
            q = point["ചോദ്യം"]
            a = point["ഉത്തരം"]
            if not q or not a:
                continue

            # Create prompt in the format you specified
            prompt = f"ചോദ്യം: {q}\nഉത്തരം:"

            # Get token IDs for the answer
            token_ids = self.get_token_ids(a)

            # pick 3 random few-shot examples
            idx = random.sample(range(len(self.list_good_shot)), 3)
            good_shots = "".join([self.list_good_shot[i] for i in idx])

            # generate 6 outputs (5 temp + 1 greedy)
            temp_generation = self.batch_generation_with_temperature(good_shots + prompt, temperature=0.5)
            greedy_gen = self.greedy_generation(good_shots + prompt)
            temp_generation.append(greedy_gen)

            # classify using Malayalam fuzzy matching
            count_know = sum([1 for temp in temp_generation if self.partial_match(a, temp)])

            # Store in the format: [prompt, answer, token_ids_list, count]
            if count_know == 6:
                knowledge_dataset.append([prompt, a, token_ids, count_know])
            elif count_know == 0:
                non_knowledge_dataset.append([prompt, a, token_ids, count_know])
            else:
                else_dataset.append([prompt, a, token_ids, count_know])

            # Update progress bar description with current counts
            pbar.set_description(
                f"K:{len(knowledge_dataset)} NK:{len(non_knowledge_dataset)} E:{len(else_dataset)}"
            )

            # periodic saving (without disrupting progress bar)
            if index % save_every == 0:
                self.save_data(knowledge_dataset, os.path.join(self.save_dir, f"{self.model_name}_{self.dataset_name}_knowledge.json"))
                self.save_data(non_knowledge_dataset, os.path.join(self.save_dir, f"{self.model_name}_{self.dataset_name}_non_knowledge.json"))
                self.save_data(else_dataset, os.path.join(self.save_dir, f"{self.model_name}_{self.dataset_name}_else.json"))
                # Add a postfix to show last save point
                pbar.set_postfix({"last_save": f"idx_{index}"})

        # Close progress bar
        pbar.close()

        # final save
        print("\n" + "=" * 50)
        print("FINAL RESULTS:")
        print(f"  Knowledge dataset: {len(knowledge_dataset)} items")
        print(f"  Non-knowledge dataset: {len(non_knowledge_dataset)} items")
        print(f"  Else dataset: {len(else_dataset)} items")
        print("=" * 50)

        self.save_data(knowledge_dataset, os.path.join(self.save_dir, f"{self.model_name}_{self.dataset_name}_knowledge.json"))
        self.save_data(non_knowledge_dataset, os.path.join(self.save_dir, f"{self.model_name}_{self.dataset_name}_non_knowledge.json"))
        self.save_data(else_dataset, os.path.join(self.save_dir, f"{self.model_name}_{self.dataset_name}_else.json"))

    # ------------------------------
    # Save helper
    # ------------------------------
    def save_data(self, data, path):
        with open(path, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

# Usage example
if __name__ == "__main__":
    # First install tiktoken if not already installed
    # !pip install tiktoken

    # Create and run the dataset processor
    processor = KnowledgeDataset(
        model_name="gpt-4o-mini",
        dataset_path="mal_dataset.json",
        save_dir="./output/",
        dataset_name="mal_dataset"
    )

    processor.create_knowledge_dataset(save_every=50)

K:3188 NK:7910 E:3902: 100%|██████████| 15000/15000 [8:13:13<00:00,  1.97s/it]



FINAL RESULTS:
  Knowledge dataset: 3188 items
  Non-knowledge dataset: 7910 items
  Else dataset: 3902 items
